# Constraint-based sampling for metabolic flux analysis (MFA)

Here we jump into the (for now) final step in our metabolic flux analysis (MFA) pipeline. After constructing an INCA script, running it in MATLAB and reimporting the data we're now here. This example notebook will guide you through different ways to integrate your MFA results into COBRA models and how to make them more reliable.

In [ ]:
import pandas as pd
import numpy as np
import time
import ast
import sys
import escher
import cobra
from tabulate import tabulate
from gurobipy import Model as GRBModel
import re
from BFAIR.INCA import INCA_reimport
from BFAIR.INCA.sampling import *

#### INCA re-import

First, let's reimport the data using our `BFAIR INCA_reimport` tools

The data is taken from a different folder

In [2]:
%cd /Users/matmat/Documents/Random data/Stefano
filename = 'data/MFA_modelInputsData/TestFile.mat'

/Users/matmat/Documents/Random data/Stefano


Then we move back to our example folder to get the remaining input

In [3]:
%cd /Users/matmat/Documents/GitHub/AutoFlow-OmicsDataHandling/examples
#filename = 'data/MFA_modelInputsData/TestFile.mat'
simulation_info = pd.read_csv('data/MFA_modelInputsData/Re-import/experimentalMS_data_I.csv')
simulation_id = 'WTEColi_113C80_U13C20_01'

/Users/matmat/Documents/GitHub/AutoFlow-OmicsDataHandling/examples


Here we re-import the INCA output

In [4]:
reimport_data = INCA_reimport()
(fittedData,
 fittedFluxes,
 fittedFragments,
 fittedMeasuredFluxes,
 fittedMeasuredFragments,
 fittedMeasuredFluxResiduals,
 fittedMeasuredFragmentResiduals,
 simulationParameters) = reimport_data.reimport(
    filename,
    simulation_info,
    simulation_id
)

Here we import the model

In [5]:
model = cobra.io.load_json_model('data/FIA_MS_example/database_files/iJO1366.json')

In [6]:
original_solution = model.optimize()
original_solution

,fluxes,reduced_costs
EX_cm_e,0.000000,0.000000e+00
EX_cmp_e,0.000000,-2.965572e-01
EX_co2_e,19.675223,0.000000e+00
EX_cobalt2_e,-0.000025,-0.000000e+00
DM_4crsol_c,0.000219,0.000000e+00
...,...,...
RNDR4,0.000000,-2.073827e-03
RNDR4b,0.000000,-2.073827e-03
RNTR1c2,0.025705,-8.673617e-18
RNTR2c2,0.026541,-8.673617e-18


Let's see what happens now when we add our new bound constraints

In [7]:
model = add_constraints(model, fittedFluxes)
model.optimize()

--- start ---
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpvjy4i66n.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpvnk3r0mw.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
add_constraints takes 0h: 0min: 10sec to run
--- end ---


/Users/matmat/opt/anaconda3/envs/bfair/lib/python3.8/site-packages/cobra/util/solver.py:430: UserWarning: solver status is 'infeasible'
  warn("solver status is '{}'".format(status), UserWarning)


<Solution infeasible at 0x7fdb9e760340>

Oh... That's not good... Well that sucks, seems like we have to deal with an infeasible solution. There are two straight forward ways: exclusion and relaxation

#### Dealing with infeasible solutions - exclusion

The easier way to deal with this issue is to simply exclude the constraints that render a model infeasible. We can do that by adding the calculated bounds one by one. If we come across a reaction whose bounds cause trouble, we restart the process and skip this one. This might have to be done a few times to exclude all troublemakers. the `add_feasible_constraints()` functions takes care of that for us. Let's reset the model first.

In [8]:
model = cobra.io.load_json_model('data/FIA_MS_example/database_files/iJO1366.json')

In [9]:
find_biomass_reaction(model)

['BIOMASS_Ec_iJO1366_WT_53p95M', 'BIOMASS_Ec_iJO1366_core_53p95M']

In [10]:
min_val = get_min_solution_val(fittedFluxes, biomass_string='Biomass')

In [11]:
fittedFluxes = replace_biomass_rxn_name(fittedFluxes, biomass_string='Biomass', biomass_rxn_name='BIOMASS_Ec_iJO1366_core_53p95M')

In [12]:
model, problems = add_feasible_constraints(model, fittedFluxes, min_val=min_val)

--- start ---
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpcewr208v.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp58ue9dsx.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Solution infeasible if adding ASPTA
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmppqvjpq2a.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp2lmxbpvh.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Solution infeasible if adding DAPDC
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp9lffjg87.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp_zzv05

The `model` is our newly constrained model and the problematic reactions can be listed in `problems`.

In [13]:
model

Name,iJO1366
Memory address,0x07fdba21d6160
Number of metabolites,1805
Number of reactions,2583
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iJO1366_core_53p95M - 1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1
Compartments,"cytosol, extracellular space, periplasm"


In [14]:
problems

['ASPTA', 'DAPDC', 'BIOMASS_Ec_iJO1366_core_53p95M', 'GLNS', 'RPI']

Now let's see what an effect these new bounds had on the predicted growth rate (the objective value) of our model

In [15]:
new_bounds_solution = model.optimize()
new_bounds_solution

,fluxes,reduced_costs
EX_cm_e,0.000000,0.000000e+00
EX_cmp_e,0.000000,-2.146363e-01
EX_co2_e,22.825015,0.000000e+00
EX_cobalt2_e,-0.000019,-0.000000e+00
DM_4crsol_c,0.000166,0.000000e+00
...,...,...
RNDR4,0.000000,-1.275067e-02
RNDR4b,0.000000,-1.275067e-02
RNTR1c2,0.019444,1.387779e-17
RNTR2c2,0.020076,1.387779e-17


And here's the star of the show, our sampling method. We trust our models because... we have to! And because smart people that knew what they were doing set them up. So in order to gain more confidence in our MFA data, we sample the model after adding the calculated bound for some of the reactions and re-calculate the fluxes a number of time. Then, we take the mean and take that as the most trustworthy calculated flux. These fluxes can be visualized, for example in tools like `Escher`

In [16]:
sampled_fluxes = cobra.sampling.sample(model, n=100, processes=2)

Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpax6p1xr1.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpk1002isq.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros


In [17]:
sampled_fluxes

,EX_cm_e,EX_cmp_e,EX_co2_e,EX_cobalt2_e,DM_4crsol_c,DM_5drib_c,DM_aacald_c,DM_amob_c,DM_mththf_c,EX_colipa_e,...,UREAtex,RNDR2b,UREAtpp,RNDR3,RNDR3b,RNDR4,RNDR4b,RNTR1c2,RNTR2c2,RNTR3c2
0,0.0,0.0,16.018131,-7.303244e-07,6.704140e-06,0.002057,0.0,6.012681e-08,0.036852,0.001194,...,-0.019839,0.007967,-0.019839,0.011892,0.003935,0.013234,0.003932,0.059038,0.056951,0.016344
1,0.0,0.0,16.376223,-7.650863e-07,7.033782e-06,0.002053,0.0,6.308325e-08,0.033813,0.000125,...,-0.002995,0.007938,-0.002995,0.011848,0.003921,0.005509,0.003917,0.079379,0.060371,0.016324
2,0.0,0.0,16.761019,-7.830306e-07,7.199207e-06,0.002031,0.0,6.456688e-08,0.020105,0.000125,...,-0.003270,0.007937,-0.003270,0.011847,0.003920,0.020103,0.002934,0.075708,0.070531,0.045136
3,0.0,0.0,16.918895,-3.368818e-07,3.070617e-06,0.008354,0.0,2.753917e-08,0.049613,0.000121,...,-0.003262,0.008063,-0.003262,0.011829,0.003914,0.020076,0.011591,0.057882,0.065022,0.039401
4,0.0,0.0,24.018016,-3.649814e-07,3.338192e-06,0.008249,0.0,2.993894e-08,0.028324,0.000108,...,-0.003083,0.007819,-0.003083,0.011557,0.003795,0.020894,0.011422,0.059064,0.066763,0.038413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,0.0,28.585133,-1.958249e-07,1.798438e-06,0.000959,0.0,1.612949e-08,0.033256,0.000050,...,-0.064054,0.109400,-0.064054,0.006827,0.009743,0.018413,0.001831,0.027056,0.321157,0.002622
96,0.0,0.0,28.640772,-7.430766e-08,6.693500e-07,0.001036,0.0,6.003139e-09,0.036334,0.000962,...,-0.067230,0.109329,-0.067230,0.006819,0.009737,0.018847,0.001829,0.041931,0.329273,0.014904
97,0.0,0.0,28.399169,-8.194506e-08,7.426063e-07,0.001039,0.0,6.660146e-09,0.036465,0.000982,...,-0.067278,0.109638,-0.067278,0.006853,0.009764,0.025415,0.001837,0.045573,0.333612,0.014922
98,0.0,0.0,28.544873,-3.422202e-07,3.132856e-06,0.001041,0.0,2.809736e-08,0.036456,0.000070,...,-0.054922,0.109571,-0.054922,0.006985,0.009758,0.025542,0.001835,0.044748,0.311016,0.015059


In [18]:
fluxes_sampling = reshape_fluxes_escher(sampled_fluxes)

In [19]:
sampled_flux_map = escher.Builder('e_coli_core.Core metabolism',
                                  reaction_data = fluxes_sampling).display_in_notebook()
sampled_flux_map

There are also other ways to cosolidate the MFA calculations and the constraint based flux predictions. One of these is lMOMA (linear Minimization Of Metabolic Adjustment). MOMA assumes that the fluxes before and after adding the new constraints should be similar, so it aims to predicting an optimum for the newly constrined model while keeping the differences to the original model small. We suggest using pFBA (parsimonious Flux Balance Analysis) instead of regular FBA for this step as pFBA aims to keep the overal fluxes low.

In [20]:
model = cobra.io.load_json_model('data/FIA_MS_example/database_files/iJO1366.json')
model, problems = add_feasible_constraints(model, fittedFluxes, min_val=min_val)

--- start ---
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpj2jgiuz3.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp81m8ya9j.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Solution infeasible if adding ASPTA
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpbduy_4rw.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros


/Users/matmat/opt/anaconda3/envs/bfair/lib/python3.8/site-packages/cobra/util/solver.py:430: UserWarning: solver status is 'infeasible'
  warn("solver status is '{}'".format(status), UserWarning)


Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpn25fjyyk.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Solution infeasible if adding DAPDC
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp1lvx11oj.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp8bxd6bje.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Solution infeasible if adding BIOMASS_Ec_iJO1366_core_53p95M
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp70zdfmo_.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp7kallcot.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Solution infeasible if adding GLNS
Read LP format model from file /var/folders

In [21]:
pfba_solution = cobra.flux_analysis.pfba(model)

In [22]:
moma_after_MFA = cobra.flux_analysis.moma(
    model=model, solution=pfba_solution, linear=True)

In [23]:
moma_after_MFA.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"]

0.7430984745775968

In [24]:
room_after_MFA = cobra.flux_analysis.room(
    model=model, solution=pfba_solution, linear=True, delta=0.03, epsilon=0.001)

In [25]:
room_after_MFA.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"]

0.7430984745776016

In [26]:
results = [['Before adding new constraints:', round(original_solution.objective_value, 2)],
           ['New constraints relaxed FBA:', round(new_bounds_solution.objective_value, 2)],
           ['New constraints relaxed pFBA:', round(pfba_solution.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"], 2)],
           ['New constraints relaxed MOMA:', round(moma_after_MFA.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"], 2)],
           ['New constraints relaxed ROOM:', round(room_after_MFA.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"], 2)]]
print(tabulate(results, headers=["Method", "Biomass function"]))

Method                            Biomass function
------------------------------  ------------------
Before adding new constraints:                0.98
New constraints relaxed FBA:                  0.74
New constraints relaxed pFBA:                 0.74
New constraints relaxed MOMA:                 0.74
New constraints relaxed ROOM:                 0.74


Of course also the MOMA results can be visualized with `Escher`. The `reshape_fluxes_escher()` can take both pandas DataFrames or cobra solutions as an input.

In [27]:
fluxes_moma = reshape_fluxes_escher(moma_after_MFA)
moma_flux_map = escher.Builder('e_coli_core.Core metabolism',
                               reaction_data = fluxes_moma).display_in_notebook()
moma_flux_map

In [28]:
fluxes_room = reshape_fluxes_escher(room_after_MFA)
room_flux_map = escher.Builder('e_coli_core.Core metabolism',
                               reaction_data = fluxes_room).display_in_notebook()
room_flux_map

#### Dealing with infeasible solutions - relaxation

Another way of dealing with infeasible models is to relax the added constraints to the point that it works again. You will need to have the Gurobi solver installed for this. The same principle is used in the `BFAIR thermo` tools. For that we add our constraints to a model that will now be infeasible. Have I meantioned that this is much more elegant, better and that you should do that? It is. The other method is *fine* but you exclude reactions and, in general, it is always better to use as much as possible of the information that is available to you.

In [29]:
model = cobra.io.load_json_model('data/FIA_MS_example/database_files/iJO1366.json')

In [30]:
model = add_constraints(model, fittedFluxes)

--- start ---
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmprnzta7j3.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpja9ccbyi.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
add_constraints takes 0h: 0min: 10sec to run
--- end ---


In [31]:
model.optimize()

<Solution infeasible at 0x7fdb38063400>

Then we make use of the handy `bound_relaxation()` function that will test our model to figure out which of these added bounds need to be adjusted and return a DataFrame that describes the affected functions and the gravity of the suggested changes. If we allow this function to be `desctructive` it will adjust the input model right away.

In [32]:
bound_relaxation(model, fittedFluxes, destructive=True, fluxes_to_ignore=['BIOMASS_Ec_iJO1366_core_53p95M'])

--- start ---
bound_relaxation takes 0h: 0min: 0sec to run
--- end ---


,lb_change,ub_change,subsystem
reaction,,,
ASPTA,-3.349375,0.000000,Alanine and Aspartate Metabolism
DAPDC_reverse_d3ab8,-0.040213,0.000000,Threonine and Lysine Metabolism
GLNS_reverse_59581,-0.767431,0.000000,Glutamate Metabolism
MTHFC_reverse_f6fcc,-0.515687,0.000000,Folate Metabolism
MTHFD_reverse_c10fd,-0.515687,0.000000,Folate Metabolism
RPI_reverse_853a1,0.000000,1.578062,Pentose Phosphate Pathway


Let's see if our model is feasible now.

In [33]:
relaxed_solution = model.optimize()
relaxed_solution

,fluxes,reduced_costs
EX_cm_e,0.000000,0.0
EX_cmp_e,0.000000,-0.0
EX_co2_e,20.755531,0.0
EX_cobalt2_e,-0.000017,-0.0
DM_4crsol_c,0.000156,0.0
...,...,...
RNDR4,0.000000,0.0
RNDR4b,0.000000,0.0
RNTR1c2,0.018316,0.0
RNTR2c2,0.018912,0.0


In [34]:
relaxed_solution.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"]

0.7

And now we're also at the point where we can let the power of constraint based models work for us and make use of the tools mentioned above in order to adjust the fluxes calculated using MFA so that they nicely fit into our model.

In [35]:
relaxed_sampled_fluxes = cobra.sampling.sample(model, n=100, processes=2)

Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpx_uq849o.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpvyzama86.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros


In [36]:
relaxed_fluxes_sampling = reshape_fluxes_escher(relaxed_sampled_fluxes)

In [37]:
relaxed_flux_map = escher.Builder('e_coli_core.Core metabolism',
                                  reaction_data = relaxed_fluxes_sampling).display_in_notebook()
relaxed_flux_map

And as before, we can use alternative optimization algorithms here, like MOMA or ROOM.

In [38]:
model = cobra.io.load_json_model('data/FIA_MS_example/database_files/iJO1366.json')
model, problems = add_feasible_constraints(model, fittedFluxes, min_val=min_val)

--- start ---
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpqs0y8efh.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpm0yerb_4.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Solution infeasible if adding ASPTA
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp1fb3_yjk.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros


/Users/matmat/opt/anaconda3/envs/bfair/lib/python3.8/site-packages/cobra/util/solver.py:430: UserWarning: solver status is 'infeasible'
  warn("solver status is '{}'".format(status), UserWarning)


Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmppcsq07o_.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Solution infeasible if adding DAPDC
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpn1namb1v.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpwwood4ix.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Solution infeasible if adding BIOMASS_Ec_iJO1366_core_53p95M
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp143p3zxk.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpycp7ith9.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Solution infeasible if adding GLNS
Read LP format model from file /var/folders

In [40]:
pfba_relaxed_solution = cobra.flux_analysis.pfba(model)

In [42]:
moma_after_relaxed_MFA = cobra.flux_analysis.moma(
    model=model, solution=pfba_relaxed_solution, linear=True)

In [43]:
moma_after_relaxed_MFA.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"]

0.7430984745776043

In [44]:
room_after_relaxed_MFA = cobra.flux_analysis.room(
    model=model, solution=pfba_relaxed_solution, linear=True, delta=0.03, epsilon=0.001)

In [45]:
room_after_relaxed_MFA.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"]

0.7430984745776043

In [46]:
from tabulate import tabulate
results = [['Before adding new constraints:', round(original_solution.objective_value, 2)],
           ['New constraints relaxed FBA:', round(relaxed_solution.objective_value, 2)],
           ['New constraints relaxed pFBA:', round(pfba_relaxed_solution.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"], 2)],
           ['New constraints relaxed MOMA:', round(moma_after_relaxed_MFA.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"], 2)],
           ['New constraints relaxed ROOM:', round(room_after_relaxed_MFA.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"], 2)]]
print(tabulate(results, headers=["Method", "Biomass function"]))

Method                            Biomass function
------------------------------  ------------------
Before adding new constraints:                0.98
New constraints relaxed FBA:                  0.7
New constraints relaxed pFBA:                 0.74
New constraints relaxed MOMA:                 0.74
New constraints relaxed ROOM:                 0.74


In [47]:
fluxes_relaxed_moma = reshape_fluxes_escher(moma_after_relaxed_MFA)
moma_relaxed_flux_map = escher.Builder('e_coli_core.Core metabolism',
                                       reaction_data = fluxes_relaxed_moma).display_in_notebook()
moma_relaxed_flux_map

In [48]:
fluxes_relaxed_room = reshape_fluxes_escher(room_after_relaxed_MFA)
room_relaxed_flux_map = escher.Builder('e_coli_core.Core metabolism',
                                       reaction_data = fluxes_relaxed_room).display_in_notebook()
room_relaxed_flux_map